In [7]:
import plotly.express as px
import pandas
import time
import random

In [17]:
class Node(object):
    def __init__(self, order):
        self.order = order
        self.keys = []
        self.values = []
        self.leaf = True

    def add(self, key, value):
        if not self.keys:
            self.keys.append(key)
            self.values.append([value])
            return None

        for i, item in enumerate(self.keys):
            if key == item:
                self.values[i].append(value)
                break

            elif key < item:
                self.keys = self.keys[:i] + [key] + self.keys[i:]
                self.values = self.values[:i] + [[value]] + self.values[i:]
                break

            elif i + 1 == len(self.keys):
                self.keys.append(key)
                self.values.append([value])

    def split(self):
        left = Node(self.order)
        right = Node(self.order)
        mid = self.order // 2

        left.keys = self.keys[:mid]
        left.values = self.values[:mid]

        right.keys = self.keys[mid:]
        right.values = self.values[mid:]

        self.keys = [right.keys[0]]
        self.values = [left, right]
        self.leaf = False

    def is_full(self):
        return len(self.keys) == self.order

    def show(self, counter=0):
        print(counter, str(self.keys))

        if not self.leaf:
            for item in self.values:
                item.show(counter + 1)
    
    def show_reverse(self, counter=0):
        print(counter, str(self.keys))

        cnt = self.values.reverse()
        if not self.leaf:
            for item in cnt:
                item.show_reverse(counter + 1)


class BPlusTree(object):
    def __init__(self, order=8):
        self.root = Node(order)

    def _find(self, node, key):
        for i, item in enumerate(node.keys):
            if key < item:
                return node.values[i], i

        return node.values[i + 1], i + 1

    def _merge(self, parent, child, index):
        parent.values.pop(index)
        pivot = child.keys[0]

        for i, item in enumerate(parent.keys):
            if pivot < item:
                parent.keys = parent.keys[:i] + [pivot] + parent.keys[i:]
                parent.values = parent.values[:i] + child.values + parent.values[i:]
                break

            elif i + 1 == len(parent.keys):
                parent.keys += [pivot]
                parent.values += child.values
                break

    def insert(self, key, value):
        parent = None
        child = self.root

        while not child.leaf:
            parent = child
            child, index = self._find(child, key)

        child.add(key, value)

        if child.is_full():
            child.split()

            if parent and not parent.is_full():
                self._merge(parent, child, index)

    def _find_2(self, node, key):
        for i, item in enumerate(node.keys):
            if key < item:
                return node.values[i], i

        return node.values[i], i

                
    def retrieve(self, key):

        child = self.root

        while not child.leaf:
            child, index = self._find(child, key)

        for i, item in enumerate(child.keys):
            if key == item:
                return child.values[i]

        while not child.leaf:
            child, index = self._find_2(child, key)

        for i, item in enumerate(child.keys):
            if key < item:
                return child.values[i - 1]

        for i, item in enumerate(child.keys):
            if key > item:
                return child.values[-1]


    def show(self):
        self.root.show()
        
    def show_reverse(self):
        self.root.show_reverse()

In [18]:
b_plus_tree = BPlusTree(order=5)
for i in range(20):
    num = random.randint(0, 180)
    b_plus_tree.insert(num, num)

b_plus_tree.show_reverse()

cnt = random.randint(0, 180)
print(cnt)
print(b_plus_tree.retrieve(cnt))



# def experimental_evaluation_data_b_plus_tree(repetitions):
#     chart_data = []

#     for i in range(repetitions):
#         current_tree = BPlusTree(5)
#         current_node_count = 100 + i * 10000
#         for j in range(current_node_count):
#             num = random.randint(0, 180)
#             current_tree.insert(num, j)

#         start_time = time.time()
#         for _ in range(100 + i * 10000):
#             current_tree.retrieve(random.randint(0, 180))
#         duration = time.time() - start_time

#         chart_data.append(dict(size=current_node_count, time=duration))
#     return chart_data

0 [46, 67, 85, 125, 144]


TypeError: 'NoneType' object is not iterable

In [6]:
# draw_chart(experimental_evaluation_data_b_plus_tree(20))

In [8]:
def draw_chart(chart_data):
    chart_data.sort(key=lambda d: d['size'])
    print(chart_data)
    fig = px.line(chart_data, x="size", y="time")
    fig.show()